In [ ]:
conda create -n snowflake python==3.10
conda activate snowflake

In [149]:
!pip install --upgrade numexpr
!pip install --upgrade bottleneck
!pip install --upgrade scikit-learn

In [140]:
import modin.pandas as pd
import os
import snowflake.connector as sc
from snowflake.snowpark.session import Session



private_key_file = '...'
private_key_file_pwd = '...'

conn_params = {
    'account': '...',
    'user': '...',
    'role': 'ACCOUNTADMIN',
    'private_key_file': private_key_file,
    'private_key_file_pwd': private_key_file_pwd,
    'database': 'HEALTH_APPLICATION',
    'schema': 'PUBLIC'
}

file_path = '...'

ctx = sc.connect(**conn_params)

cursor = ctx.cursor()
list_files_query = f"LIST @HEALTH_APPLICATION_DATASETS;"
cursor.execute(list_files_query)
files = cursor.fetchall()
print(files)
for i in files:
    file = i[0]
    get_query = f'GET @{file} file://{file_path}'
    cursor.execute(get_query)
cursor.close()

[('health_application_datasets/LGHC.csv', 8574, 'aff6b621576d71614e38fc4bc540c14a', 'Fri, 20 Dec 2024 23:33:39 GMT'), ('health_application_datasets/US_CDI.csv', 88872126, None, 'Fri, 20 Dec 2024 08:30:15 GMT')]


True

In [141]:
import numpy as np
import os

os.chdir(file_path)

file1 = files[0][0]
file1 = os.path.split(file)[1]

file2 = files[1][0]
file2 = os.path.split(file)[1]

In [142]:
import modin.pandas as pd
df1 = pd.read_csv(f'{file1}')
diabetes_df1 = df1[df1['Question'].str.contains('diabetes', case=False, na=False)]
df2 = pd.read_csv(f'{file2}')
diabetes_df2 = df2[df2['Question'].str.contains('diabetes', case=False, na=False)]
diabetes_df = pd.concat([diabetes_df1, diabetes_df2])
features = features = ['LocationDesc', 'StratificationID1','StratificationCategoryID1', 'DataValue']
diabetes_df = diabetes_df[features].dropna()
diabetes_df

,LocationDesc,StratificationID1,StratificationCategoryID1,DataValue
3,Maine,SEXM,SEX,11.4
9,Alabama,AGE1844,AGE,5.0
21,Vermont,SEXM,SEX,9.3
22,Virgin Islands,SEXM,SEX,12.0
29,Massachusetts,SEXM,SEX,9.2
...,...,...,...,...
311660,Utah,HIS,RACE,11.0
311670,Virginia,OVR,OVERALL,12.8
311678,Washington,SEXF,SEX,9.2
311705,Virgin Islands,HIS,RACE,9.6


In [143]:
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import joblib

categorical_cols = ['LocationDesc', 'StratificationID1', 'StratificationCategoryID1']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    diabetes_df[col] = le.fit_transform(diabetes_df[col])
    label_encoders[col] = le
    
X = diabetes_df.drop(columns=['DataValue']) 
y = diabetes_df['DataValue']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

joblib.dump(rf_model, 'chronic_illness_diabetes_predictor.pkl')
joblib.dump(scaler, "scaler.pkl")
joblib.dump(label_encoders, "label_encoders.pkl")

y_pred = rf_model.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")
r2 = r2_score(y_test, y_pred)
print(f'R-squared (R²): {r2:.2f}')


Root Mean Squared Error (RMSE): 10251.93
Mean Absolute Error (MAE): 1705.04
R-squared (R²): 0.20


In [144]:
new_user_data = {
    'LocationDesc': 'Georgia', 
    'StratificationID1': 'SEXM', 
    'StratificationCategoryID1': 'SEX', 
}

new_user_df = pd.DataFrame([new_user_data])
for col in categorical_cols:
    new_user_df[col] = label_encoders[col].transform(new_user_df[col])
new_user_df = scaler.transform(new_user_df) 
diabetes_risk = rf_model.predict(new_user_df)  
print(f"Predicted likelihood of developing diabetes: {diabetes_risk[0]:.2f}%")

Predicted likelihood of developing diabetes: 846.74%
